A few things to try:
-just include experimental condition as covariate when sctransforming merged dataset
    -ie https://www.biostars.org/p/492222/

-use wt reps as references and integrate and do label transfer
    https://satijalab.org/seurat/archive/v3.0/integration.html

-use approach from Col_h3h3 script to integrate SCT t
    
overall goal is to identify AZ cells

-test strategy below with Che_wei's filtered data

-Test:
IT v CW filtering
SCT with ot without batch correction
SCT with ot without experiment correction

check number of AZ cells in each case
make DE list for each

-New idea from 1/5/22:
   *analyze/integrate only WT. 
   *Identify AZ cluster by measure of known marker gene specificty across leiden clustering resolutions (categorical cross-entropy)
   *Identify marker genes for putative AZ cluster not reduced in bulk data
   *separately analyze/integrate mutant
   *Identify AZ cluster by measure of de novo marker gene specificity across leiden clustering resolutions (same as above)
   *Compare pseudo-bulk
   

In [ ]:
library(here)
library(Matrix)
library(tidyverse)
library(Seurat)
library(ggpointdensity)
library(scales)
library(DropletUtils)
library(reticulate)
library(repr)
#library(DoubletFinder)
library(future)
library(RColorBrewer)
library(edgeR)
options(future.globals.maxSize = 100000 * 1024^2)

proto_genes=read.csv("../data/bulk_data/protoplasting.csv")
proto_list=as.character(proto_genes[abs(proto_genes$logFC) > 1,]$genes)

In [2]:
# THIS IS THE PREPROCESSING TO GET TO THE INTEGRATED SEURAT OBJECT. JUST UNCOMMENT ENTIRE CELL.

#IWT FILTERING
wt_1_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_26_combined.rds")
wt_2_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_67.rds")
mut_1_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_27_combined.rds")
mut_2_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_68.rds")
YFP_1_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_101.rds")
YFP_2_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_103.rds")
KE_1_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_102.rds")
KE_2_seu = readRDS(file = "../data/seurat_objects/seurat_raw_3_11_21/sc_104.rds")

#set experimental condition
wt_1_seu@meta.data$condition = "wt_unsorted"
wt_2_seu@meta.data$condition = "wt_unsorted"
mut_1_seu@meta.data$condition = "mut_unsorted"
mut_2_seu@meta.data$condition = "mut_unsorted"
YFP_1_seu@meta.data$condition = "wt_sorted"
YFP_2_seu@meta.data$condition = "wt_sorted"
KE_1_seu@meta.data$condition = "mut_sorted"
KE_2_seu@meta.data$condition = "mut_sorted"

#set batch
wt_1_seu@meta.data$batch = "1"
wt_2_seu@meta.data$batch = "2"
mut_1_seu@meta.data$batch  = "1"
mut_2_seu@meta.data$batch  = "2"
YFP_1_seu@meta.data$batch  = "3"
YFP_2_seu@meta.data$batch  = "3"
KE_1_seu@meta.data$batch = "3"
KE_2_seu@meta.data$batch = "3"

In [3]:
counts1 <- as.matrix(GetAssayData(subset(wt_1_seu), slot = "counts"))
counts2 <- as.matrix(GetAssayData(subset(wt_2_seu), slot = "counts"))
counts3 <- as.matrix(GetAssayData(subset(mut_1_seu), slot = "counts"))
counts4 <- as.matrix(GetAssayData(subset(mut_2_seu), slot = "counts"))
counts5 <- as.matrix(GetAssayData(subset(YFP_1_seu), slot = "counts"))
counts6 <- as.matrix(GetAssayData(subset(YFP_2_seu), slot = "counts"))
counts7 <- as.matrix(GetAssayData(subset(KE_1_seu), slot = "counts"))
counts8 <- as.matrix(GetAssayData(subset(KE_2_seu), slot = "counts"))

In [4]:
mean(Matrix::colSums(counts1 > 0))
mean(Matrix::colSums(counts2 > 0))
mean(Matrix::colSums(counts3 > 0))
mean(Matrix::colSums(counts4 > 0))
mean(Matrix::colSums(counts5 > 0))
mean(Matrix::colSums(counts6 > 0))
mean(Matrix::colSums(counts7 > 0))
mean(Matrix::colSums(counts8 > 0))

[1] 2203.144

[1] 2417.368

[1] 1226.755

[1] 2365.174

[1] 1699.96

[1] 1779.033

[1] 1947.698

[1] 1349.222

In [ ]:
#WT
wt_1_seu = SCTransform(wt_1_seu)
wt_2_seu = SCTransform(wt_2_seu)
YFP_1_seu = SCTransform(YFP_1_seu)
YFP_2_seu = SCTransform(YFP_2_seu)

In [ ]:
seu_intd_wt = seu_integrate(wt_1_seu, wt_2_seu, YFP_1_seu, YFP_2_seu, filename = "1_6_22_IWT", nfeatures = 3000)

In [ ]:
#h3h3
mut_1_seu = SCTransform(mut_1_seu)
mut_2_seu = SCTransform(mut_2_seu)
KE_1_seu = SCTransform(KE_1_seu)
KE_2_seu = SCTransform(KE_2_seu)

In [ ]:
seu_intd = seu_integrate(wt_1_seu, wt_2_seu, mut_1_seu, mut_2_seu, YFP_1_seu, YFP_2_seu, KE_1_seu, KE_2_seu, filename = "6_15_21_IWT", nfeatures = 3000)

In [ ]:
seu_intd = readRDS(file = "../data/intd_seu_objects/6_15_21_IWT.rds")

In [ ]:
seu_intd

In [ ]:
DefaultAssay(seu_intd) <- "integrated"
options(repr.plot.width=12, repr.plot.height=12)
# Run the standard workflow for visualization and clustering
#all_intd_sct <- ScaleData(all_intd_sct, verbose = FALSE)
seu_intd <- RunPCA(seu_intd, npcs = 50, verbose = FALSE, approx = FALSE)
#From RunPCA doc: Features to compute PCA on. If features=NULL, PCA will be run using the variable features for the Assay. 
#Note that the features must be present in the scaled data. Any requested features that are not scaled or have 0 variance 
#will be dropped, and the PCA will be run using the remaining features.

seu_intd <- FindNeighbors(seu_intd, dims = 1:50)
seu_intd <- FindClusters(seu_intd, resolution = 0.5)
seu_intd <- RunUMAP(seu_intd, reduction = "pca", dims = 1:50)
DimPlot(seu_intd, reduction = "umap", label = TRUE)

In [ ]:
options(repr.plot.width=30, repr.plot.height=8)
DimPlot(seu_intd, reduction = "umap", label = TRUE, split.by = "orig.ident")

In [ ]:
cluster = "5"
seu_intd@active.assay = "RNA"

wt_1_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_26_combined"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_26_combined"), ident = cluster)]))
wt_2_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_67"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_67"), ident = cluster)]))
mut_1_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_27_combined"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_27_combined"), ident = cluster)]))
mut_2_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_68"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_68"), ident = cluster)]))
YFP_1_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_101"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_101"), ident = cluster)]))
YFP_2_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_103"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_103"), ident = cluster)]))
KE_1_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_102"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_102"), ident = cluster)]))
KE_2_AZ <- rowSums(as.matrix(GetAssayData(subset(seu_intd, subset = orig.ident == "sc_104"), slot = "counts")[, WhichCells(subset(seu_intd, subset = orig.ident == "sc_104"), ident = cluster)]))



In [ ]:
all_AZ = cbind(wt_1_AZ, wt_2_AZ, mut_1_AZ, mut_2_AZ, YFP_1_AZ, YFP_2_AZ, KE_1_AZ, KE_2_AZ)
colnames(all_AZ) = c("wt_1_pb", "wt_2_pb", "mut_1_pb", "mut_2_pb", "YFP_1_pb", "YFP_2_pb", "KE_1_pb", "KE_2_pb")

In [ ]:
head(all_AZ)

In [ ]:
all_AZ_bad_norm = all_AZ
all_AZ_bad_norm[,1] = all_AZ[,1]/colSums(all_AZ)[1] 
all_AZ_bad_norm[,2] = all_AZ[,2]/colSums(all_AZ)[2] 
all_AZ_bad_norm[,3] = all_AZ[,3]/colSums(all_AZ)[3] 
all_AZ_bad_norm[,4] = all_AZ[,4]/colSums(all_AZ)[4] 
all_AZ_bad_norm[,5] = all_AZ[,5]/colSums(all_AZ)[5] 
all_AZ_bad_norm[,6] = all_AZ[,6]/colSums(all_AZ)[6] 
all_AZ_bad_norm[,7] = all_AZ[,7]/colSums(all_AZ)[7] 
all_AZ_bad_norm[,8] = all_AZ[,8]/colSums(all_AZ)[8] 
all_AZ_bad_norm = all_AZ_bad_norm * 1000000

In [ ]:
all_AZ_bad_norm["AT3G59850",]

In [ ]:
head(all_AZ_bad_norm)

In [ ]:
DefaultAssay(seu_intd) <- "SCT"

#haesa
FeaturePlot(seu_intd, features = c("AT4G28490"),  pt.size = 1.5, order = TRUE, min = .5, split.by = "orig.ident")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")


In [ ]:
DefaultAssay(seu_intd) <- "SCT"

#haesa
FeaturePlot(seu_intd, features = c("AT4G28490"),  pt.size = 1.5, order = TRUE, min = .5, split.by = "orig.ident")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

#qrt2
FeaturePlot(seu_intd, features = c("AT3G07970"),   pt.size = 1, order = TRUE)# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")


#pgazat
FeaturePlot(seu_intd, features = c("AT2G41850"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

#myb62
FeaturePlot(seu_intd, features = c("AT1G68320"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

#ppg
FeaturePlot(seu_intd, features = c("AT3G59850"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

#gdsl
FeaturePlot(seu_intd, features = c("AT5G03820"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

In [ ]:
seu_intd

In [ ]:
#ALL SAMPLES (INCLUDING SORTED) MERGED AND SCT PERFORMED
#merge
merged = merge(wt_1_seu, y = c(wt_2_seu, mut_1_seu, mut_2_seu, YFP_1_seu, YFP_2_seu, KE_1_seu, KE_2_seu), project = "abscission")

#set features for SCR
features = rownames(merged@assays$RNA@counts)
features = features[!(features%in%proto_list)]

ab_sct = SCTransform(merged, vars.to.regress = c("condition", "percent.mito") , residual.features =  features, verbose = TRUE)

In [ ]:
ab_sct <- RunPCA(ab_sct, verbose = FALSE)
ab_sct <- RunUMAP(ab_sct, dims = 1:50, verbose = FALSE)

ab_sct <- FindNeighbors(ab_sct, dims = 1:50, verbose = FALSE)
ab_sct <- FindClusters(ab_sct, verbose = FALSE)

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
DimPlot(ab_sct, label = TRUE) + NoLegend()

In [ ]:
saveRDS(ab_sct, file = paste("../data/seurat_objects/seurat_raw_3_11_21/", "ab_sct_5_19_21", ".rds", sep=""))

In [ ]:
DefaultAssay(ab_sct) = "SCT"
options(repr.plot.width=20, repr.plot.height=8)
Idents(ab_sct) <- "condition"
FeaturePlot(ab_sct, features = "nCount_RNA", split.by = "condition") 

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)
#haesa
FeaturePlot(ab_sct, features = c("AT4G28490"),  pt.size = 1, order = TRUE, split.by = "condition", min = 1)# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

#qrt2
FeaturePlot(ab_sct, features = c("AT3G07970"),   pt.size = 1, order = TRUE, split.by = "geno", min = 1)# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")


#pgazat
FeaturePlot(ab_sct, features = c("AT2G41850"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

#myb62
FeaturePlot(ab_sct, features = c("AT1G68320"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

#ppg
FeaturePlot(ab_sct, features = c("AT3G59850"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")

#gdsl
FeaturePlot(ab_sct, features = c("AT5G03820"),  pt.size = 1, order = TRUE)#, split.by = "geno")#, slot = "data")


In [ ]:
Idents(ab_sct) <- "experiment"
DimPlot(ab_sct, reduction = "umap", split.by = "experiment")

In [ ]:
#Just WT samples , sorted and unsorted
merged_wt = merge(wt_1_seu, y = c(wt_2_seu, YFP_1_seu, YFP_2_seu), project = "abscission")

features = rownames(merged@assays$RNA@counts)
features = features[!(features%in%proto_list)]

wt_sct = SCTransform(merged_wt, vars.to.regress = c("experiment", "percent.mito") , residual.features =  features, verbose = TRUE)
saveRDS(wt_sct, file = paste("../data/seurat_objects/seurat_raw_3_11_21/", "wt_sct_5_13_21", ".rds", sep=""))

In [ ]:
wt_sct <- RunPCA(wt_sct, verbose = FALSE)
wt_sct<- RunUMAP(wt_sct, dims = 1:50, verbose = FALSE)

wt_sct <- FindNeighbors(wt_sct, dims = 1:50, verbose = FALSE)
wt_sct <- FindClusters(wt_sct, verbose = FALSE)

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
DimPlot(wt_sct, label = TRUE) + NoLegend()

In [ ]:
Idents(wt_sct) <- "experiment"
DimPlot(wt_sct, label = TRUE) + NoLegend()

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)
#haesa
FeaturePlot(wt_sct, features = c("AT4G28490"),  pt.size = 1, order = TRUE, split.by = "experiment")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

#qrt2
FeaturePlot(wt_sct, features = c("AT3G07970"),   pt.size = 1, order = TRUE, split.by = "experiment")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")


#pgazat
FeaturePlot(wt_sct, features = c("AT2G41850"),  pt.size = 1, order = TRUE, split.by = "experiment")#, split.by = "geno")#, slot = "data")

#myb62
FeaturePlot(wt_sct, features = c("AT1G68320"),  pt.size = 1, order = TRUE, split.by = "experiment")#, split.by = "geno")#, slot = "data")

#ppg
FeaturePlot(wt_sct, features = c("AT3G59850"),  pt.size = 1, order = TRUE, split.by = "experiment")#, split.by = "geno")#, slot = "data")

#gdsl
FeaturePlot(wt_sct, features = c("AT5G03820"),  pt.size = 1, order = TRUE, split.by = "experiment")#, split.by = "geno")#, slot = "data")

#ida
FeaturePlot(wt_sct, features = c("AT1G68765"),  pt.size = 1, order = TRUE, split.by = "experiment")#, split.by = "geno")#, slot = "data")


In [ ]:
#WT and mutant unsorted. IWT FILTERING
merged = merge(wt_1_seu, y = c(wt_2_seu, mut_1_seu, mut_2_seu), project = "abscission")

features = rownames(merged@assays$RNA@counts)
features = features[!(features%in%proto_list)]

unsorted_sct = SCTransform(merged, vars.to.regress = c("condition", "percent.mito") , residual.features =  features, verbose = TRUE)

In [ ]:
unsorted_sct <- RunPCA(unsorted_sct, verbose = FALSE)
unsorted_sct<- RunUMAP(unsorted_sct, dims = 1:50, verbose = FALSE)

unsorted_sct <- FindNeighbors(unsorted_sct, dims = 1:50, verbose = FALSE)
unsorted_sct <- FindClusters(unsorted_sct, verbose = FALSE)

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
DimPlot(unsorted_sct, label = TRUE) + NoLegend()

In [ ]:
DefaultAssay(unsorted_sct) = "SCT"

options(repr.plot.width=12, repr.plot.height=12)
#haesa
FeaturePlot(unsorted_sct, features = c("AT4G28490"),  pt.size = 2, order = TRUE, split.by = "orig.ident")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

#qrt2
FeaturePlot(unsorted_sct, features = c("AT3G07970"),  pt.size = 1, order = TRUE, split.by = "condition")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

#qrt2
FeaturePlot(unsorted_sct, features = c("AT2G18020"),  pt.size = 1, order = TRUE, split.by = "condition")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

FeaturePlot(unsorted_sct, features = c("AT1G34430"),  pt.size = 1, order = TRUE, split.by = "condition")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")





In [ ]:
saveRDS(unsorted_sct, file = paste("../data/seurat_objects/seurat_raw_3_11_21/", "unsorted_sct_5_19_21", ".rds", sep=""))

In [ ]:
cw_unsorted = readRDS("~/data/buckets/single_cell_bucket_3_4_21/Flower_4S_seu3.rds")

In [ ]:
cw_list = SplitObject(cw_unsorted, split.by = "orig.ident")

In [ ]:
#rename split objects
wt_1_seu = cw_list[[1]]
wt_2_seu = cw_list[[3]]
mut_1_seu = cw_list[[2]]
mut_2_seu = cw_list[[4]]


#set experimental condition
wt_1_seu@meta.data$condition = "wt_unsorted"
wt_2_seu@meta.data$condition = "wt_unsorted"
mut_1_seu@meta.data$condition = "mut_unsorted"
mut_2_seu@meta.data$condition = "mut_unsorted"


In [ ]:
#WT and mutant unsorted. CW FILTERING
merged = merge(wt_1_seu, y = c(wt_2_seu, mut_1_seu, mut_2_seu), project = "abscission")

features = rownames(merged@assays$RNA@counts)
features = features[!(features%in%proto_list)]

unsorted_sct_cw = SCTransform(merged, vars.to.regress = c("condition", "percent.mt") , residual.features =  features, verbose = TRUE)

In [ ]:
unsorted_sct_cw <- RunPCA(unsorted_sct_cw, verbose = FALSE)
unsorted_sct_cw<- RunUMAP(unsorted_sct_cw, dims = 1:50, verbose = FALSE)

unsorted_sct_cw <- FindNeighbors(unsorted_sct_cw, dims = 1:50, verbose = FALSE)
unsorted_sct_cw <- FindClusters(unsorted_sct_cw, verbose = FALSE)

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
DimPlot(unsorted_sct_cw, label = TRUE) + NoLegend()

In [ ]:
options(repr.plot.width=24, repr.plot.height=6)
FeaturePlot(unsorted_sct_cw, features = c("AT4G28490"),  pt.size = 1, order = TRUE, split.by = "orig.ident")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")


In [ ]:
saveRDS(unsorted_sct_cw, file = paste("../data/seurat_objects/seurat_raw_3_11_21/", "unsorted_sct_cw_5_20_21", ".rds", sep=""))

In [ ]:
#IWT FILTERING, unsorted, regress batch and condition
#WT and mutant unsorted. IWT FILTERING
merged = merge(wt_1_seu, y = c(wt_2_seu, mut_1_seu, mut_2_seu), project = "abscission")

features = rownames(merged@assays$RNA@counts)
features = features[!(features%in%proto_list)]

unsorted_sct_batch = SCTransform(merged, vars.to.regress = c("condition", "percent.mito", "batch") , residual.features =  features, verbose = TRUE)

In [ ]:
unsorted_sct_batch  <- RunPCA(unsorted_sct_batch , verbose = FALSE)
unsorted_sct_batch <- RunUMAP(unsorted_sct_batch , dims = 1:50, verbose = FALSE)

unsorted_sct_batch  <- FindNeighbors(unsorted_sct_batch , dims = 1:50, verbose = FALSE)
unsorted_sct_batch  <- FindClusters(unsorted_sct_batch , verbose = FALSE)

options(repr.plot.width=12, repr.plot.height=12)
DimPlot(unsorted_sct_batch, label = TRUE) + NoLegend()

In [ ]:
options(repr.plot.width=24, repr.plot.height=6)
FeaturePlot(unsorted_sct_batch, features = c("AT4G28490"),  pt.size = 1, order = TRUE, split.by = "orig.ident")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

In [ ]:
saveRDS(unsorted_sct_batch, file = paste("../data/seurat_objects/seurat_raw_3_11_21/", "unsorted_sct_batch_5_20_21", ".rds", sep=""))

In [ ]:
#IWT FILTERING, unsorted, regress batch and condition
#WT and mutant unsorted. IWT FILTERING
merged = merge(wt_1_seu, y = c(wt_2_seu, mut_1_seu, mut_2_seu), project = "abscission")

features = rownames(merged@assays$RNA@counts)
features = features[!(features%in%proto_list)]

unsorted_sct_batch_only = SCTransform(merged, vars.to.regress = c("percent.mito", "batch") , residual.features =  features, verbose = TRUE)

In [ ]:
unsorted_sct_batch_only  <- RunPCA(unsorted_sct_batch_only , verbose = FALSE)
unsorted_sct_batch_only <- RunUMAP(unsorted_sct_batch_only , dims = 1:50, verbose = FALSE)

unsorted_sct_batch_only  <- FindNeighbors(unsorted_sct_batch_only , dims = 1:50, verbose = FALSE)
unsorted_sct_batch_only  <- FindClusters(unsorted_sct_batch_only , verbose = FALSE)

options(repr.plot.width=12, repr.plot.height=12)
DimPlot(unsorted_sct_batch_only, label = TRUE) + NoLegend()

In [ ]:
options(repr.plot.width=24, repr.plot.height=6)
FeaturePlot(unsorted_sct_batch_only, features = c("AT4G28490"),  pt.size = 1, order = TRUE, split.by = "orig.ident")# + scale_colour_gradient(low = "gray94", high = "red", space = "Lab", na.value = "grey50", guide = "colourbar",aesthetics = "colour")

In [ ]:
sum(Idents(unsorted_sct_batch_only) == "16")
sum(Idents(unsorted_sct_batch) == "16")
sum(Idents(unsorted_sct_cw) == "16")

In [ ]:
saveRDS(unsorted_sct_batch_only, file = paste("../data/seurat_objects/seurat_raw_3_11_21/", "unsorted_sct_batch_only_5_20_21", ".rds", sep=""))

In [ ]:
#takes a list of Seurat objects with SCT transform run
seu_integrate <- function(..., filename, nfeatures){
    seu.list <- list(...) # THIS WILL BE A LIST STORING EVERYTHING:
    
    ref.genes = rownames(seu.list[[1]]@assays$RNA)
    assay_list <- rep("SCT", length(seu.list))
# loop
#    for (i in 1:length(seu.list)) {
#         seu.list[[i]]@assays$spliced_RNA <- NULL
#         seu.list[[i]]@assays$spliced_SCT <- NULL
#         seu.list[[i]]@assays$unspliced_RNA <- NULL
#         seu.list[[i]]@assays$unspliced_SCT <- NULL
        

    # integration
    rc.features <- SelectIntegrationFeatures(object.list = seu.list, nfeatures = nfeatures)
    rc.features <- rc.features[(!c(grepl("ATMG",rc.features) | grepl("ATCG",rc.features) | rc.features%in%proto_list))]
    
    seu.list <- PrepSCTIntegration(object.list = seu.list, anchor.features = rc.features, verbose = TRUE, assay = assay_list)
    seu.list <- lapply(X = seu.list, FUN = RunPCA, verbose = FALSE, features = rc.features)
    rc.anchors <- FindIntegrationAnchors(object.list = seu.list, normalization.method = "SCT", anchor.features = rc.features, verbose = TRUE, reference=1, reduction = "rpca")
    
    to_integrate <- Reduce(intersect, lapply(rc.anchors@object.list, rownames))
    # integrate data and keep full geneset
       
    rc.integrated <- IntegrateData(anchorset = rc.anchors,  features.to.integrate = to_integrate, normalization.method = "SCT", verbose = TRUE)
    rc.integrated <- RunPCA(rc.integrated, npcs = 50, verbose = FALSE, approx = FALSE)
    
    #save object    
    saveRDS(rc.integrated, file = paste("../data/intd_seu_objects/",filename,".rds", sep = ""))
    return(rc.integrated)
#    }
}